In [35]:
from pyspark import SparkConf, SparkContext
import numpy as np
import os
import time
sc.stop()
conf = SparkConf().setMaster("local").setAppName("LSH")
sc = SparkContext(conf=conf)

In [36]:
import random

hash_function = []
prime_table = [2 , 3 , 5 , 7 , 11 , 13 , 17 , 19 , 23 , 29 , 31 , 37 , 41 , 43 , 47 , 53 , 59 , 61 , 67 , 71 , 73 , 79 , 83 , 89 , 97]
for i in range(0,100):
    hash_function.append((random.randint(1,10000)))

In [37]:
def splitTextToTriplet(string):
    words = string.split()
    grouped_words = [' '.join(words[i: i + 3]) for i in range(0, len(words)-3)]
    return grouped_words

In [38]:
data = sc.textFile("/Users/haoweichung/Desktop/清大/Senior/data_mining/HW4/001.txt").filter(lambda x: x != '')
for i in range(2,102):
    data1 = sc.textFile("/Users/haoweichung/Desktop/清大/Senior/data_mining/HW4/"+ '{:03}'.format(i) + ".txt").filter(lambda x: x != '')
    data = sc.union([data,data1])

In [39]:
matrix = data.flatMap(splitTextToTriplet).zipWithIndex().groupByKey().mapValues(list).map(lambda x: (x[0],x[1][0]))

# RDD 

matrix 記錄了所有shingle word對應到的shingle id
* splitTextToTriplet: 這個mapper會把每篇文章拆成三個三個字一組組成一個新的RDD，接著zipWithIndex會把每一個shingle word進行編號，方便待會hashing，在最後用一個mapper把rdd轉為(shingle_word, shingle_id)的樣子

* (shingle_word, shingle_id)

In [40]:
data = sc.textFile("/Users/haoweichung/Desktop/清大/Senior/data_mining/HW4/001.txt").filter(lambda x: x != '').flatMap(splitTextToTriplet).map(lambda x: (1,x))
for i in range(2,102):
    data1 = sc.textFile("/Users/haoweichung/Desktop/清大/Senior/data_mining/HW4/"+ '{:03}'.format(i) + ".txt").filter(lambda x: x != '').flatMap(splitTextToTriplet).map(lambda x: (i,x))
    data = sc.union([data,data1])

在這邊的for迴圈會把所有的文章逐一讀出來並全部union在一起，並且利用filter把換行濾掉，接著再用map把rdd轉為 (document_id, shingle words)

In [41]:
shingle_matrix = data.map(lambda x : (x[1],x[0])).join(matrix).flatMap(lambda x : [x[1]])

# RDD

第一個map會把key與value調換，把shingle word放在key，再來用join把data跟matrix合併，再用flatMap留下(doc_id, shingle_id)待會可以做min_hash

* (doc_id, shingle_id)

In [42]:
def min_hash(x):
    list_ = []
    for i in range(100):
        list_.append(((x[0],i),((((i+1)*x[1])+hash_function[i])%99371)%25883))
    return list_

def sign2(x):
    return (min(x[1]),(x[0][0],x[0][1]))

def to_candidate(x):
    list_ = []
    for i in range(len(x[1])):
        for j in range(i+1, len(x[1])):
            if x[1][i][0] > x[1][j][0]:
                list_.append((x[1][i][0], (x[1][j][0], x[1][j][1])))
            else:
                list_.append((x[1][j][0], (x[1][i][0], x[1][j][1])))
    return list_

def to_candidate2(x):
    list_ = []
    check = [0]*102
    for i in range(len(x[1])):
        for j in range(i+1, len(x[1])):
            if (x[1][i][0] == x[1][j][0]) and (check[x[1][i][0]] == 0):
                if (x[1][i][1] == x[1][j][1] + 1) and (x[1][j][1] % 2 == 0) :
                    check[x[1][i][0]] = 1
                    list_.append((x[0], x[1][i][0]))
                elif (x[1][i][1] + 1 == x[1][j][1]) and (x[1][i][1] % 2 == 0) :
                    check[x[1][i][0]] = 1
                    list_.append((x[0], x[1][i][0]))
                    
    return list_

In [43]:
signature_matrix = shingle_matrix.flatMap(min_hash).groupByKey().mapValues(list).map(sign2)

# RDD

* min_hash會把rdd轉成 ((shingle_id, hash_id), hash_value)，接者group by key以list的方式把value存在後面，map(sign2)會把rdd中list最小值作為key，這邊的最小值其實就會是該document在hash_id這個function下的min_hash value，這麼一來就可以做出(hash_value, (doc_id, hash_id))的rdd資料結構了。

* sign2 -> (hash_value, (doc_id, hash_id))

In [53]:
pair_matrix = signature_matrix.groupByKey().mapValues(list).flatMap(to_candidate).groupByKey().mapValues(list).flatMap(to_candidate2).collect()

In [54]:
print(pair_matrix)

[(3, 3), (3, 2), (4, 3), (4, 2), (5, 3), (5, 4), (5, 2), (6, 3), (6, 4), (6, 2), (7, 4), (7, 3), (8, 3), (8, 8), (8, 2), (8, 7), (8, 4), (8, 5), (8, 6), (9, 3), (9, 8), (9, 9), (9, 2), (9, 4), (9, 5), (9, 7), (9, 6), (10, 9), (10, 8), (10, 3), (10, 10), (10, 5), (10, 4), (10, 7), (10, 2), (11, 9), (11, 3), (11, 7), (11, 8), (11, 2), (11, 10), (11, 11), (11, 5), (12, 3), (12, 2), (12, 9), (12, 11), (12, 6), (12, 12), (12, 4), (12, 8), (12, 7), (13, 11), (13, 3), (13, 8), (13, 7), (13, 10), (13, 12), (13, 13), (13, 4), (13, 9), (13, 6), (13, 5), (13, 2), (14, 10), (14, 11), (14, 3), (14, 9), (14, 8), (14, 14), (14, 12), (14, 2), (14, 4), (14, 7), (14, 13), (14, 6), (14, 1), (15, 10), (15, 9), (15, 13), (15, 8), (15, 14), (15, 7), (16, 14), (16, 9), (16, 10), (16, 8), (16, 3), (16, 2), (16, 4), (16, 13), (16, 6), (16, 16), (16, 15), (16, 5), (16, 12), (16, 11), (17, 14), (17, 3), (17, 4), (17, 13), (17, 8), (17, 17), (17, 7), (17, 6), (17, 10), (17, 16), (17, 12), (17, 15), (17, 11), (18,

# RDD

* 接著利用同樣的key，也就是同樣的hash value，我們可以把所有有一樣hash value的document中的某一個shingle id全部放到同一個rdd內，並且用to_candidate map成 (較大的document_id, (較小的document id, hash_id)。接著再groupByKey，(doc_id(largest), [(doc_id1, hash_id),(doc_id2, hash_id),(doc_id3, hash_id),(doc_id4, hash_id),(doc_id5, hash_id) .....]。to_candidate2會把bucket內部有hash到的pair append到list裡面 return成新的rdd代表所有的candidate pair。接著collect起來，待會用來計算similarity

* signature_matrix.groupByKey().mapValues(list).flatMap(to_candidate)

* .flatMap(to_candidate2).collect()
* (document1, document2) where doc_id1 > doc_id2
* already found all candidate pairs

In [45]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))
    

def find_sim(x):
    list_ = []
    sorted_pair = sorted(x[1])
    print(len(sorted_pair))
    for i in pair_matrix:
        if not(i[0]-1 == i[1]-1):
            list_.append((jaccard_similarity(sorted_pair[i[0]-1][1], sorted_pair[i[1]-1][1]),sorted_pair[i[0]-1][0],sorted_pair[i[1]-1][0]))
    return list_
        
        

In [46]:
t = shingle_matrix.groupByKey().mapValues(list).map(lambda x: (1, x)).groupByKey().mapValues(list).flatMap(find_sim)

先把shingle group起來得到(doc_id, ([shingle1, shingle2 .....]) 接者用一個map把所有element改成(1, x)這樣可以把所有的element groupByKey就可以把全部的element放在一起。接著用find_sim找每個candidate pair的jaccard similarity，結果rdd為((doc_id, doc_id), sim)

In [47]:
final = sorted(t.collect(), reverse=True)

In [48]:
print("document1   document2   jaccard simularity:")
for i in range(10):
    vec = final[i]
    print('{0:2d}         {1:3d}          {2:4f}'.format(vec[2], vec[1], vec[0]))


document1   document2   jaccard simularity:
52          84          1.000000
12          20          1.000000
47          49          0.759676
30          35          0.710977
49          88          0.517730
48          49          0.483412
23          38          0.482260
14          40          0.402239
47          88          0.393531
47          48          0.367237
